# Artificial Neural Netword

## Importing Libraries

In [8]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import tensorflow as tf 

## Data Preprocessing

### Importing Dataset

In [6]:
dataset = pd.read_csv('../../datasets/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values 

In [10]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Label Encoding the "Gender" Column 

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [12]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the "Geography" Column

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [15]:
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


### Splitting the dataset into the training set and test set

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature scaling

In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initializing the ANN

In [19]:
ann = tf.keras.models.Sequential()

### Adding the first hidden layer

In [20]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [22]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Training the ANN

### Compiling the ANN

In [23]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [24]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5268 - accuracy: 0.7629
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4657 - accuracy: 0.7969
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4462 - accuracy: 0.8025
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.8075
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4278 - accuracy: 0.8138
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.8192
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4110 - accuracy: 0.8238
Epoch 8/100
250/250 [==============================] - 0s 937us/step - loss: 0.3988 - accuracy: 0.8324
Epoch 9/100
250/250 [==============================] - 0s 956us/step - loss: 0.3866 - accuracy: 0.8425
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3754 - ac

## Making the predictions 

### Predicting the result of a single observation

In [33]:
x_pred = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]])
x_pred[:, 2] = le.transform(x_pred[:, 2])
x_pred = np.array(ct.transform(x_pred))
x_pred = sc.transform(x_pred)

x_pred[0]

array([ 0.98560362, -0.5698444 , -0.57369368, -0.52111599,  0.91601335,
        0.10961719, -0.68538967, -0.2569057 ,  0.8095029 ,  0.64259497,
        0.9687384 , -0.87203322])

In [35]:
print(ann.predict(x_pred) > 0.5)

[[False]]


### Predicting the results for the test set

In [37]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Printing the model confusion matrix and accuracy

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)

print(cm)
accuracy_score(y_test, y_pred)

[[1512   83]
 [ 195  210]]


0.861